##  **Install The Library And Load Data From the Cloud**

In [ ]:
import pandas as pd
Import pandas as np
from google.colab import drive
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
!pip install scikit-surprise
# Mount Google Drive
drive.mount('/content/drive')

# Save the dataset to Google Drive
rating.to_csv('/content/drive/My Drive/rating.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
rating = pd.read_csv('rating.csv')

In [ ]:
movie = pd.read_csv('movie.csv')
df = pd.merge(movie,rating, how="inner", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41


In [ ]:
#picking randomly 4 movie and their id 541
movie_ids = [130219, 356, 4422, 541]
movies = ["The Dark Knight (2011)",
          "Cries and Whispers (Viskningar och rop) (1972)",
          "Forrest Gump (1994)",
          "Blade Runner (1982)"]

In [ ]:
#downgrading our data set, since too much
sample_df = df[df.movieId.isin(movie_ids)]
sample_df.head()


,movieId,title,genres,userId,rating,timestamp
1380063,200,"Tie That Binds, The (1995)",Thriller,165,2.0,1996-06-22 23:10:32
1380064,200,"Tie That Binds, The (1995)",Thriller,928,2.0,1996-08-02 10:53:00
1380065,200,"Tie That Binds, The (1995)",Thriller,1233,5.0,2007-12-23 12:50:53
1380066,200,"Tie That Binds, The (1995)",Thriller,1931,3.0,1996-11-23 18:30:51
1380067,200,"Tie That Binds, The (1995)",Thriller,1947,5.0,1996-03-28 15:05:42


In [ ]:
sample_df.shape

(97343, 6)

In [ ]:
# Rows = users, Columns = movies
user_movie_df = sample_df.pivot_table(index=["userId"],
                                      columns=["title"],
                                      values="rating")


In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
#We are converting it into a suitable data format for Surprise:
data = Dataset.load_from_df(sample_df[['userId','movieId','rating']], reader)


In [ ]:
#Modelling phase
trainset, testset = train_test_split(data, test_size=.25)
svd_model = SVD()
svd_model.fit(trainset)
predictions = svd_model.test(testset)

In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9526


0.9525896228582339

In [ ]:
svd_model.predict(uid=1.0, iid=541, verbose=True)

user: 1.0        item: 541        r_ui = None   est = 4.02   {'was_impossible': False}


Prediction(uid=1.0, iid=541, r_ui=None, est=4.021740425278456, details={'was_impossible': False})

In [ ]:
sample_df[sample_df["userId"] == 1]

,movieId,title,genres,userId,rating,timestamp


our model predicted **4.22**

but orginally it is 4.00 which is good accuracy

In [ ]:
def suggest(df,user_id,sug):

    didnt_watch = df["movieId"][~(df["userId"] == user_id)].drop_duplicates().values.tolist()
    temp_dict={}

    for i in didnt_watch:

        temp_dict[i] = svd_model.predict(uid=user_id, iid=i)[3]

    suggestions = pd.DataFrame(temp_dict.items(),columns=["movieId",'possible_rate']).sort_values(by="possible_rate", ascending=False).head(sug)
    merged = pd.merge(suggestions,movie[["movieId","title"]], how="inner", on="movieId")

    return merged

In [ ]:
suggest(df,15,50).sort_values(by="title", ascending=False).head(10)

,movieId,possible_rate,title
20,89674,4.102815,Zombie Island Massacre (1984)
44,89800,4.102815,You're Telling Me! (1934)
29,89872,4.102815,Warrior of the Lost World (1983)
49,89774,4.102815,Warrior (2011)
13,1,4.102815,Toy Story (1995)
3,89753,4.102815,Tinker Tailor Soldier Spy (2011)
4,89747,4.102815,Taxi! (1932)
10,89718,4.102815,Sweet Karma (2009)
39,89829,4.102815,"Star Witness, The (1931)"
27,89765,4.102815,"Sleep, My Love (1948)"


In [ ]:
suggest(df,5,50).sort_values(by="title", ascending=False).head(10)
